In [2]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 09
### Team Member Names: Aryan Singh, Jack Smith, and Samyak Jain
### Team Strategy Chosen: Market Meet (Market Beat, Market Meet, Risk-Free)

### Ticker Data:

# Stock Filter:
This cell filters the initial CSV file of tickers. We remove any stocks that don't trade in USD or CAD and exclude stocks with average daily volume below 5,000 shares between October 1, 2024 and September 30, 2025 excluding months with less that 18 trading days. This filter also removes any delisted or fake tickers from the list.

ADD Duplicate ticker remover and any cross listed companies(only keep one of them).

In [7]:
start_date = '2024-10-01'
end_date = '2025-09-30'
min_volume = 5000
min_trading_days=18

#tickers_df = pd.read_csv('Tickers_Example.csv', header=None, names=['Ticker'])
tickers_df = pd.DataFrame({
    'Ticker': [
        'AAPL', 'MSFT', 'NVDA', 'INTC', 'CSCO',
        'JPM', 'BAC', 'WFC', 'GS', 'MS',
        'CAT', 'BA', 'UPS', 'HON', 'GE',
        'XOM', 'CVX', 'COP', 'SLB', 'EOG',
        'AMZN', 'TSLA', 'HD', 'NKE', 'MCD',
        'GOOGL', 'META', 'DIS', 'NFLX', 'VZ',
        'LIN', 'APD', 'NEM', 'FCX', 'DOW',
         'WMT', 'PG', 'KO', 'PEP', 'COST',
        'JNJ', 'UNH', 'PFE', 'ABBV', 'MRK',
        'NEE', 'DUK', 'SO', 'D', 'AEP',
        'AMT', 'PLD', 'EQIX', 'SPG', 'SPG']})
    

tickers = tickers_df['Ticker'].tolist()

filtered_tickers = []
seen = set()

for ticker in tickers:
    if ticker in seen:
        print(f"[DUPLICATE] {ticker}: Duplicate ticker skipped.")
        continue

    if not is_us_or_ca_ticker_by_currency(ticker):
        print(f"[REMOVED] {ticker}: Not USD or CAD currency.")
        continue

    ticker_data = yf.Ticker(ticker)
    hist = ticker_data.history(start=start_date, end=end_date)

    if isinstance(hist.index, pd.DatetimeIndex) and hist.index.tz is not None:
        hist.index = hist.index.tz_convert(None)

    if hist.empty:
        print(f"[DELISTED] {ticker}: No historical data returned.")
        continue

    if 'Volume' not in hist.columns:
        print(f"[REMOVED] {ticker}: No Volume column in stock data.")
        continue

    avg_vol = average_daily_volume_adjusted(hist)

    if avg_vol < min_volume:
        print(f"[REMOVED] {ticker}: Avg volume {avg_vol:.2f} < {min_volume}.")
        continue

    seen.add(ticker)              
    filtered_tickers.append(ticker) 


[DUPLICATE] SPG: Duplicate ticker skipped.


['AAPL',
 'MSFT',
 'NVDA',
 'INTC',
 'CSCO',
 'JPM',
 'BAC',
 'WFC',
 'GS',
 'MS',
 'CAT',
 'BA',
 'UPS',
 'HON',
 'GE',
 'XOM',
 'CVX',
 'COP',
 'SLB',
 'EOG',
 'AMZN',
 'TSLA',
 'HD',
 'NKE',
 'MCD',
 'GOOGL',
 'META',
 'DIS',
 'NFLX',
 'VZ',
 'LIN',
 'APD',
 'NEM',
 'FCX',
 'DOW',
 'WMT',
 'PG',
 'KO',
 'PEP',
 'COST',
 'JNJ',
 'UNH',
 'PFE',
 'ABBV',
 'MRK',
 'NEE',
 'DUK',
 'SO',
 'D',
 'AEP',
 'AMT',
 'PLD',
 'EQIX',
 'SPG']

### Optimal Benchmark Weighting On Different Sectors:

TSX Composite weighting comes from https://investingnews.com/daily/resource-investing/how-is-the-sptsx-composite-index-weighted/

S&P500 Weighting comes from: https://www.schwab.com/learn/story/stock-sector-outlook

Note that taking the equally weighted average of the weightings results in a total weight of 99.95%. To fix this, we simply added 0.05% to the Financial Services sector as it is the most correlated sector to the benchmark.

In the end, we want our portfolio weightings per sector to be similar to the benchmark. When discussing Industry diversification in Topic 8(portfolio Approach), we learned that companies in the same sector or industry tend to move together because they react similarly to market forces. This idea can be applied to reason that, if our final portfolio has similar weightings to the benchmark per sector, our portfolio would likely move in a similar way to the benchmark as well.

In this section, we created a dictionary of sectors and their weightings in the benchmark, so that the proper weightings can be applied to our final portfolio later.

In [3]:
# what the benchmark weighting on different sectors should be

bench_weights = {
    'Financial Services': 0.2275,
    'Technology': 0.198,
    'Industrials': 0.1115,
    'Energy': 0.1035,
    'Consumer Cyclical': 0.0705,
    'Communication Services': 0.0635,
    'Basic Materials': 0.072,
    'Consumer Defensive': 0.0515,
    'Healthcare': 0.0495,
    'Utilities': 0.0315,
    'Real Estate': 0.021
}

### DataFrame To Store The Necessary Information (Ticker, Sector, Price, Currency, And Market Cap):

This section creates a DataFrame containing each stock's ticker, sector, current price(in its trading currency), trading currency, and market cap in CAD. We get this information from Yfinance. The currency and market cap data are important for later calculations involving our portfolio construction and weighting, as well as meeting the portfolio constraints outlined in the assignment instructions. We convert all market caps to CAD, as our portfolio constraints are set in CAD, and so we can accurately compare the market cap of both US and Canadian companies by having all market caps in CAD. Stocks with unknown sectors are removed since we cannot properly allocate them according to our benchmark weighting.

In [4]:
initial_capital = 1_000_000

def get_usd_to_cad_rate():
    
        fx = yf.Ticker("CADUSD=X")
        fx_hist = fx.history(period="1d")
        rate_cadusd = fx_hist["Close"].iloc[-1]
        return 1 / rate_cadusd

usd_to_cad = get_usd_to_cad_rate()

tickers_list = []
sectors_list = []
prices_list = []
currencies_list = []
market_caps_list = []

for ticker in filtered_tickers:
    
    try:
        stock = yf.Ticker(ticker)
        
        info = stock.info or {}
        fast = stock.fast_info or {}
        sector = info.get('sector', 'Unknown')
        
        
        if sector == 'Unknown':
            print(f"[REMOVED] {ticker}: Unknown Sector '{sector}'")
            continue
        
        hist = stock.history(period='1d')
        
        if hist.empty or 'Close' not in hist.columns:
            print(f"[REMOVED] {ticker}: No Price Data.")
            continue

        price = float(hist['Close'].iloc[-1])
        
        currency = fast.get('currency') or info.get('currency') or 'USD'
        if currency not in ['USD', 'CAD']:
            currency = 'USD'
        
        market_cap = fast.get('market_cap', info.get('marketCap', 0))
        
        if market_cap is None or pd.isna(market_cap) or market_cap <= 0:
            market_cap = 0
        
        if currency == 'USD':
            market_cap_cad = market_cap * usd_to_cad
        else:
            market_cap_cad = market_cap
        tickers_list.append(ticker)
        sectors_list.append(sector)
        prices_list.append(price)
        currencies_list.append(currency)
        market_caps_list.append(market_cap_cad)
    
    except Exception as e:
        print(f"Skipped {ticker}: {e}")
        continue

df = pd.DataFrame({
    'Ticker': tickers_list,
    'Sector': sectors_list,
    'Price': prices_list,
    'Currency': currencies_list, 
    'Market Cap': market_caps_list
})

print(df.to_markdown())
print(f"\nSector By Size:\n{df.groupby('Sector').size()}")


|    | Ticker   | Sector                 |   Price | Currency   |   Market Cap |
|---:|:---------|:-----------------------|--------:|:-----------|-------------:|
|  0 | AAPL     | Technology             |  266.25 | USD        |  5.56747e+12 |
|  1 | MSFT     | Technology             |  478.43 | USD        |  5.0109e+12  |
|  2 | NVDA     | Technology             |  180.64 | USD        |  6.20734e+12 |
|  3 | INTC     | Technology             |   33.62 | USD        |  2.25964e+11 |
|  4 | CSCO     | Technology             |   75.44 | USD        |  4.19993e+11 |
|  5 | JPM      | Financial Services     |  298.38 | USD        |  1.15608e+12 |
|  6 | BAC      | Financial Services     |   51    | USD        |  5.32271e+11 |
|  7 | WFC      | Financial Services     |   82.4  | USD        |  3.71935e+11 |
|  8 | GS       | Financial Services     |  773.7  | USD        |  3.30019e+11 |
|  9 | MS       | Financial Services     |  158.7  | USD        |  3.56964e+11 |
| 10 | CAT      | Industrial

### Removing Lowest Market Cap Stocks

This cell filters out low market cap stocks by requiring a minimum market cap of $1B CAD. These stocks are excluded because they tend to have higher volatility and often do not correlate super well with the market. For our market meet strategy, we need stocks that move in correlation with the overall market, and large-cap stocks generally demonstrate stronger correlation with large indices. The code also ensures we maintain at least one small-cap stock (< 2B CAD).

In [5]:
min_small_cap = 1_000_000_000
limit_small_cap = 2_000_000_000
large_cap_cutoff = 10_000_000_000

df_filtered = df[df['Market Cap'] >= min_small_cap].copy()

small_caps = df_filtered[df_filtered['Market Cap'] < limit_small_cap]

large_caps = df_filtered[df_filtered['Market Cap'] > large_cap_cutoff]

if len(small_caps) == 0:
    
    original_small_caps = df[(df['Market Cap'] < limit_small_cap) & (df['Market Cap'] >= min_small_cap)]
    
    if not original_small_caps.empty:
        
        best_small_cap = original_small_caps.sort_values(by='Market Cap', ascending=False).iloc[0]

        df_filtered = pd.concat([df_filtered, pd.DataFrame([best_small_cap])], ignore_index=True)

if len(large_caps) == 0:
    
    original_large_caps = df[df['Market Cap'] > large_cap_cutoff]
    
    if not original_large_caps.empty:
        
        best_large_cap = original_large_caps.sort_values(by='Market Cap', ascending=False).iloc[0]
        
        df_filtered = pd.concat([df_filtered, pd.DataFrame([best_large_cap])], ignore_index=True)

df = df_filtered.reset_index(drop=True)

df


,Ticker,Sector,Price,Currency,Market Cap
0,AAPL,Technology,266.250000,USD,5.567474e+12
1,MSFT,Technology,478.429993,USD,5.010898e+12
2,NVDA,Technology,180.639999,USD,6.207340e+12
3,INTC,Technology,33.619999,USD,2.259641e+11
4,CSCO,Technology,75.440002,USD,4.199933e+11
5,JPM,Financial Services,298.380005,USD,1.156077e+12
6,BAC,Financial Services,51.000000,USD,5.322709e+11
7,WFC,Financial Services,82.400002,USD,3.719352e+11
8,GS,Financial Services,773.700012,USD,3.300187e+11
9,MS,Financial Services,158.699997,USD,3.569640e+11


### Calculating Annual Volatility:

In this section, we calculate Annual Volatility. The formula for such calculation and its importance are outlined here: https://www.fool.com/investing/how-to-calculate/annualized-volatility/

Volatility is computed as the standard deviation of daily returns multiplied by the square root of the number of trading periods. In this section, we assume 251 trading days in the 1-year period. It is important that this would inflate the volatility of stocks that have not been trading for at least 1 year. However, we deemed this okay, as for our strategy, we wish to invest in well-established stocks that have a history of following the benchmark, so inflating the volatility of stocks that have traded for less than 1 year and then removing those stocks is in line with our strategy and goal. We also remove any stocks that have traded less than 60 total days in the past year, as again, we wish to only invest in well-established stocks. We wish to remove high volatility stocks, as high volatility could cause our portfolio to deviate significantly from the benchmark. We want to remove stocks with less than 60 trading days in the last year because with that low number of trading days, the volatility of the stock becomes statistically unreliable. 

In [6]:
# calculating annual volatility

end_date = datetime.now()
start_date = datetime(end_date.year - 1, end_date.month, end_date.day)

trading_days_per_year = 251
min_days_required = 60

volatilities = []

for ticker in df['Ticker']:

    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)

        if isinstance(hist.index, pd.DatetimeIndex) and hist.index.tz is not None:
            hist.index = hist.index.tz_convert(None)

        if hist is None or hist.empty or len(hist) < min_days_required:
            volatilities.append(np.nan)
            continue

        returns = hist['Close'].pct_change().dropna()

        if returns.empty:
            volatilities.append(np.nan)
            continue

        annual_vol = returns.std() * np.sqrt(trading_days_per_year)
        volatilities.append(annual_vol)

    except Exception:
        volatilities.append(np.nan)
        continue

df['Volatility'] = volatilities

# Remove unusable tickers
df = df.dropna(subset=['Volatility']).reset_index(drop=True)

df


,Ticker,Sector,Price,Currency,Market Cap,Volatility
0,AAPL,Technology,266.250000,USD,5.567474e+12,0.325486
1,MSFT,Technology,478.429993,USD,5.010898e+12,0.243530
2,NVDA,Technology,180.639999,USD,6.207340e+12,0.498676
3,INTC,Technology,33.619999,USD,2.259641e+11,0.627878
4,CSCO,Technology,75.440002,USD,4.199933e+11,0.235453
5,JPM,Financial Services,298.380005,USD,1.156077e+12,0.242464
6,BAC,Financial Services,51.000000,USD,5.322709e+11,0.269042
7,WFC,Financial Services,82.400002,USD,3.719352e+11,0.292430
8,GS,Financial Services,773.700012,USD,3.300187e+11,0.310807
9,MS,Financial Services,158.699997,USD,3.569640e+11,0.309014


### Before Filtering High Volatile Stocks:

In [7]:
print("Before filtering:", len(df))
print(df.groupby("Sector").size())

Before filtering: 54
Sector
Basic Materials           5
Communication Services    5
Consumer Cyclical         5
Consumer Defensive        5
Energy                    5
Financial Services        5
Healthcare                5
Industrials               5
Real Estate               4
Technology                5
Utilities                 5
dtype: int64


### Removing Highest Volatility Stocks:

This section removes the top 25% most volatile stocks within each sector, keeping only the 75% with the lowest volatility. High-volatility stocks are more likely to cause our portfolio to deviate significantly from the benchmark. By filtering within each sector rather than across all eligible stocks, we can maintain sector diversification while still being able to weight sectors according to the benchmark weightings. Sectors with fewer than four stocks are left alone to preserve diversification within each sector. This approach balances risk reduction with our need for an adequate number of stocks in each sector.

In [8]:


cutoff = 0.75

filtered_rows = []
seen_sectors = []

for i in range(len(df)):
    sector_name = df.iloc[i]['Sector']
    if sector_name not in seen_sectors:
        seen_sectors.append(sector_name)

for sector in seen_sectors:
    
    sector_df = df[df['Sector'] == sector].copy()
    if len(sector_df) < 4:
        filtered_rows.append(sector_df)
        continue
    
    sector_df = sector_df.sort_values(by='Volatility').reset_index(drop=True)
        
    keep_count = int(len(sector_df) * cutoff)
    
    if keep_count < 1:
        keep_count = 1
    
    sector_kept = sector_df.iloc[:keep_count]
    filtered_rows.append(sector_kept)

df = pd.concat(filtered_rows, ignore_index=True)

df


,Ticker,Sector,Price,Currency,Market Cap,Volatility
0,CSCO,Technology,75.440002,USD,4.199933e+11,0.235453
1,MSFT,Technology,478.429993,USD,5.010898e+12,0.243530
2,AAPL,Technology,266.250000,USD,5.567474e+12,0.325486
3,JPM,Financial Services,298.380005,USD,1.156077e+12,0.242464
4,BAC,Financial Services,51.000000,USD,5.322709e+11,0.269042
5,WFC,Financial Services,82.400002,USD,3.719352e+11,0.292430
6,HON,Industrials,188.139999,USD,1.786407e+11,0.251382
7,GE,Industrials,290.619995,USD,4.342448e+11,0.306878
8,CAT,Industrials,546.130005,USD,3.619163e+11,0.313645
9,XOM,Energy,117.019997,USD,7.029491e+11,0.232928


### After Filtering High Volatile Stocks:

In [9]:
print("After filtering:", len(df))
print(df.groupby("Sector").size())

After filtering: 33
Sector
Basic Materials           3
Communication Services    3
Consumer Cyclical         3
Consumer Defensive        3
Energy                    3
Financial Services        3
Healthcare                3
Industrials               3
Real Estate               3
Technology                3
Utilities                 3
dtype: int64


### Defining the Benchmark:

In this cell, we are constructing our benchmark as an equally-weighted average of daily returns from the S&P500 and TSX Composite indices over the past year, as outlined in the assignment. We calculate daily percentage returns for both indices and average them to create a single benchmark return. This benchmark will serve as our target for correlation when selecting stocks to invest in. Aligning the dates between both indices ensures accurate return matching for correlation calculations. 

In [10]:


end_date = datetime.now()
start_date = datetime(end_date.year - 1, end_date.month, end_date.day)

tsx_symbol = "^GSPTSE"
spx_symbol = "^GSPC"

tsx_ticker = yf.Ticker(tsx_symbol)
spx_ticker = yf.Ticker(spx_symbol)

tsx_hist = tsx_ticker.history(start=start_date, end=end_date)
spx_hist = spx_ticker.history(start=start_date, end=end_date)

if isinstance(tsx_hist.index, pd.DatetimeIndex) and tsx_hist.index.tz is not None:
    tsx_hist.index = tsx_hist.index.tz_convert(None)
    
if isinstance(spx_hist.index, pd.DatetimeIndex) and spx_hist.index.tz is not None:
    spx_hist.index = spx_hist.index.tz_convert(None)
    
tsx_returns = tsx_hist['Close'].pct_change()
spx_returns = spx_hist['Close'].pct_change()

benchmark_df = (pd.DataFrame({'TSX Return': tsx_returns,'S&P 500 Return': spx_returns})).dropna()

benchmark_df['Benchmark Return'] = (benchmark_df['TSX Return'] + benchmark_df['S&P 500 Return']) / 2
benchmark_df.index = benchmark_df.index.strftime('%Y-%m-%d')

benchmark_df


,TSX Return,S&P 500 Return,Benchmark Return
Date,,,
2024-11-21,0.014147,0.005340,0.009744
2024-11-22,0.002111,0.003468,0.002790
2024-11-25,-0.001332,0.003020,0.000844
2024-11-26,-0.000209,0.005722,0.002757
2024-11-27,0.003275,-0.003801,-0.000263
...,...,...,...
2025-11-14,0.002410,-0.000502,0.000954
2025-11-17,-0.008254,-0.009162,-0.008708
2025-11-18,-0.001320,-0.008256,-0.004788


### Calculating stock-benchmark Correlations

This section calculates the correlation between each stock's daily returns and the benchmark returns over the past year. Correlation measures how closely a stock's price movements track with the benchmark. Stocks with higher correlation to the benchmark are more likely to produce returns similar to the market average. We require at least 20 overlapping trading days to ensure statistical significance of the correlation. This correlation data will be used to rank and select the best stocks for tracking the benchmark.

In [11]:
# Pick the top 1-3 stocks in each sector with the best correlation to the benchmark over the past year.

correlations = []

for i in range(len(df)):
    
    ticker = df.iloc[i]['Ticker']
    
    try:
        
        stock = yf.Ticker(ticker)
        stock_hist = stock.history(start=start_date, end=end_date)

        if stock_hist is None or stock_hist.empty:
            
            correlations.append(np.nan)
            
            continue

        stock_returns = stock_hist['Close'].pct_change().dropna()

        if stock_returns.empty:
            
            correlations.append(np.nan)
            
            continue

        stock_returns.index = stock_returns.index.strftime('%Y-%m-%d')
        
        dates = []
        
        for x in stock_returns.index:
            
            if x in benchmark_df.index:
                
                dates.append(x)

        if len(dates) < 20:
            
            correlations.append(np.nan)
            
            continue

        stock_aligned = stock_returns.loc[dates]
        bench_aligned = benchmark_df.loc[dates, 'Benchmark Return']

        corr_between_stock_and_bench = stock_aligned.corr(bench_aligned)
        
        correlations.append(corr_between_stock_and_bench)

    except Exception:
        
        correlations.append(np.nan)

### Selecting Top Stocks by Correlation

This section selects the top 3 stocks in each sector based on their correlation to the benchmark. By choosing highly correlated stocks, we maximize the probability that our portfolio will move with the benchmark. The sector-by-sector selection ensures that we maintain enough stocks in each sector to which we allocate weighting. Stocks with missing correlation data are removed.

In [12]:

df['Correlation'] = correlations
df = df.dropna(subset=['Correlation']).reset_index(drop=True)

sector_lst = []

for i in range(len(df)):
    
    sector = df.iloc[i]['Sector']
    
    if sector not in sector_lst:
        
        sector_lst.append(sector)

stocks_chosen = []

for sector in sector_lst:
    
    sector_df = df[df['Sector'] == sector].copy()
    sector_df = sector_df.dropna(subset=['Correlation'])
    
    if len(sector_df) == 0:
        
        continue
    
    sector_df = sector_df.sort_values(by='Correlation', ascending=False).reset_index(drop=True)
    
    if len(sector_df) <= 3:
        
        chosen = sector_df
        
    else:
        
        chosen = sector_df.iloc[:3]
    
    stocks_chosen.append(chosen)

stocks_chosen_df = pd.concat(stocks_chosen, ignore_index=True)

stocks_chosen_df


,Ticker,Sector,Price,Currency,Market Cap,Volatility,Correlation
0,AAPL,Technology,266.250000,USD,5.567474e+12,0.325486,0.719020
1,CSCO,Technology,75.440002,USD,4.199933e+11,0.235453,0.695719
2,MSFT,Technology,478.429993,USD,5.010898e+12,0.243530,0.660126
3,JPM,Financial Services,298.380005,USD,1.156077e+12,0.242464,0.769279
4,BAC,Financial Services,51.000000,USD,5.322709e+11,0.269042,0.717678
5,WFC,Financial Services,82.400002,USD,3.719352e+11,0.292430,0.697565
6,GE,Industrials,290.619995,USD,4.342448e+11,0.306878,0.692806
7,CAT,Industrials,546.130005,USD,3.619163e+11,0.313645,0.682380
8,HON,Industrials,188.139999,USD,1.786407e+11,0.251382,0.615338
9,EOG,Energy,106.070000,USD,8.160246e+10,0.282349,0.549221


### Limiting to 25 Stocks

This cell caps the portfolio at 25 stocks by selecting those with the highest correlation to the benchmark if there are currently more than 25 eligable stocks. We limit the portfolio to 25 stocks as per the assignment outline. The stocks are sorted by correlation in descending order to prioritize higher correlation stocks(stocks that will likely follow the benchmark better).

In [13]:

if len(stocks_chosen_df) > 25:
    
    stocks_chosen_df = stocks_chosen_df.sort_values(by="Correlation", ascending=False).reset_index(drop=True)
    stocks_chosen_df = stocks_chosen_df.iloc[:25].reset_index(drop=True)

x = len(stocks_chosen_df)

print("Number Of Stocks For Portfolio:", x)
stocks_chosen_df

Number Of Stocks For Portfolio: 25


,Ticker,Sector,Price,Currency,Market Cap,Volatility,Correlation
0,JPM,Financial Services,298.380005,USD,1.156077e+12,0.242464,0.769279
1,AAPL,Technology,266.250000,USD,5.567474e+12,0.325486,0.719020
2,BAC,Financial Services,51.000000,USD,5.322709e+11,0.269042,0.717678
3,WFC,Financial Services,82.400002,USD,3.719352e+11,0.292430,0.697565
4,CSCO,Technology,75.440002,USD,4.199933e+11,0.235453,0.695719
5,GE,Industrials,290.619995,USD,4.342448e+11,0.306878,0.692806
6,AMZN,Consumer Cyclical,217.139999,USD,3.270767e+12,0.350289,0.687248
7,DIS,Communication Services,102.699997,USD,2.601761e+11,0.293301,0.686083
8,CAT,Industrials,546.130005,USD,3.619163e+11,0.313645,0.682380
9,SPG,Real Estate,180.070007,USD,9.570716e+10,0.260538,0.670876


Testing Code:

In [14]:
# THIS CODE WAS PRIMARILY USED FOR TESTING
# In this code, it calculates the sector weights based on the total market cap of the selected stocks. It does not affect the portfolio's 
# actual weighting or share calculations.

# sector_totals = {}
# total = 0

# for i in range(len(stocks_chosen_df)):
#     sector = stocks_chosen_df.iloc[i]["Sector"]
#     marketcap = stocks_chosen_df.iloc[i]["Market Cap"]

#     if pd.isna(marketcap) or marketcap <= 0:
#         continue

#     if sector not in sector_totals:
#         sector_totals[sector] = 0.0

#     sector_totals[sector] += marketcap
#     total += marketcap

# portfolio_sector_weights = {}

# for sector in sector_totals:
#     if total > 0:
#         portfolio_sector_weights[sector] = sector_totals[sector] / total
#     else:
#         portfolio_sector_weights[sector] = 0.0

# print("Portfolio Sector Weights:")

# for sector, weight in portfolio_sector_weights.items():
#     print(f"{sector}: {weight:.4f}")


### Portfolio Weight Calculation Using Benchmark Sector Targets:

This cell calculates initial portfolio wieghts for each stock based on the benchmark sector targets and market cap within each sector. Our weight formula multiplies the benchmark's sector weight by each stock's proportional share of its sector's total market cap. This approach ensures our portfolio's sector allocation is similar to the benchmark while distributing our investment within sectors based on company size. We also apply a minimum weight of (1/2n) and max weight of 15% for each stock in our portfolio.

In [15]:

# enforcing the min. and max. weight allowed per stock
# x = number of stocks
min_weight = 1 / (2 * x)
max_weight = 0.15

# created a dict to store the total market cap of each sector
sector_market_cap_chosen = {}

# created a list to store the initial calculated weights
initial_weights = [0] * x

# created a for loop which will loop through each sector and the calculate total market cap
for i in range(x):

    # retrieving neccessary info.
    stock_sector = stocks_chosen_df.iloc[i]["Sector"]
    cap = float(stocks_chosen_df.iloc[i]["Market Cap"])

    # create a new entry if needed
    if stock_sector not in sector_market_cap_chosen:
        sector_market_cap_chosen[stock_sector] = 0

    # add market cap to sector total
    sector_market_cap_chosen[stock_sector] += cap

# created a for loop to compute the initial weights based on benchmark sector weights
for i in range(x):

    # retrieving neccessary info.
    row = stocks_chosen_df.iloc[i]
    stock_sector = row["Sector"]
    cap = row["Market Cap"]

    # benchmark weighting for the current sector
    sector_target = bench_weights.get(stock_sector, 0)

    # total sector market cap
    sector_cap_sum = sector_market_cap_chosen[stock_sector]

    # if a sector missing in benchmark or sector cap sum is zero, use equal weighted method
    if sector_target == 0 or sector_cap_sum <= 0:
        
        weight = 1 / x
        
    else:
        
        # the weight is based on benchmark sector weight * stock's share of sector
        weight = sector_target * (cap / sector_cap_sum)

    # applying the min weight and max weight rules
    weight = max(min_weight, min(weight, max_weight))

    # store the weight
    initial_weights[i] = weight

# normalizing out the weights to make sure it adds to 1
total_weight = sum(initial_weights)
initial_weights = [weight_of_sector / total_weight for weight_of_sector in initial_weights]

# calculating actual sector totals after weighting
sector_weights_actual = {}

# created a for loop to retrieve the necessary info. and add the stock's weight to its sector's total weight
for i in range(x):
    sector = stocks_chosen_df.iloc[i]["Sector"]
    sector_weights_actual[sector] = sector_weights_actual.get(sector, 0) + initial_weights[i]

# created a for loop to apply the 40% sector cap, where needed
for sector, weight_of_sector in sector_weights_actual.items():

    # if the weighting of the sector is greater than 40%, scale it accordingly
    if weight_of_sector > 0.40:
        scale = 0.40 / weight_of_sector

        # for loop to apply the scaling to each stock within that sector
        for i in range(x):
            if stocks_chosen_df.iloc[i]["Sector"] == sector:
                initial_weights[i] *= scale

# another normalization to make sure weights sum to 1 again
total_weight = sum(initial_weights)
initial_weights = [weight_of_sector / total_weight for weight_of_sector in initial_weights]


### Enforcing Weight Constriants:

This cell implements the portfolio weight rules of 40% max sector weight and 15% max individual stock weight within our portfolio. We scale down any sector exceeding 40% weight and redistribute the excess weight across the portfolio. We also cap stocks weighting at 15% and redistrubute any excess weight across the portfolio.

In [16]:
# created a list to store the sectors present in the selected stocks
sectors_present = []

# created a for loop to loop through each selected stock and find the unique sectors
for i in range(len(stocks_chosen_df)):
    sector = stocks_chosen_df.iloc[i]["Sector"]

    if sector not in sectors_present:
        sectors_present.append(sector)

# dict to count how many stocks appear in each sector
sector_counts = {}

# created a for loop to fill the sector_counts dict
for i in range(len(stocks_chosen_df)):
    sector = stocks_chosen_df.iloc[i]["Sector"]

    # add new sector if needed
    if sector not in sector_counts:
        sector_counts[sector] = 0

    # increment for the given sector
    sector_counts[sector] += 1

# dict to store each sector’s benchmark weighting
sector_targets = {}

# created a for loop to assign each sector its benchmark weight. The loop will assign the value 0 if not in bench_weights
for sector in sectors_present:
    if sector in bench_weights:
        sector_targets[sector] = bench_weights[sector]
    else:
        sector_targets[sector] = 0

# calculating the total benchmark weighting for normalization
total_target = sum(sector_targets.values())

# created an if statement to check if total > 0, normalize benchmark weights so they add to 1
if total_target > 0:
    for sector in sector_targets:
        sector_targets[sector] = sector_targets[sector] / total_target
else:
    # if something breaks, use equally-weighted method
    equal_weight = 1 / len(sectors_present)
    for sector in sector_targets:
        sector_targets[sector] = equal_weight

# list to store the initial unadjusted weight per stock
initial_weights = []

# created a for loop to loop though each sector and assign each stock the same share of its sector’s total weight
for i in range(len(stocks_chosen_df)):
    sector = stocks_chosen_df.iloc[i]["Sector"]
    target_sector_weight = sector_targets.get(sector, 0)

    # divide sector weight equally among the stocks
    per_stock_weight = target_sector_weight / sector_counts[sector]
    initial_weights.append(per_stock_weight)

# normalize weights to make sure they sum to 1
total = sum(initial_weights)

if total > 0:
    initial_weights = [w / total for w in initial_weights]
else:
    # backup
    initial_weights = [1 / len(stocks_chosen_df)] * len(stocks_chosen_df)

# copy initial weights to start applying sector caps
weights = initial_weights[:]
x = len(stocks_chosen_df)

# created a function called apply_sector_cap which will enforce the 40% maximum sector weight rule
def apply_sector_cap(weights):

    # dict to store each sector’s current weight
    sector_weight_current = {}

    # created a for loop to sum the weights for each sector
    for i in range(x):
        sector = stocks_chosen_df.iloc[i]["Sector"]
        if sector not in sector_weight_current:
            sector_weight_current[sector] = 0
        sector_weight_current[sector] += weights[i]

    # created a for loop to loop through each sector and apply scaling if a sector goes above 40%
    for sector in sector_weight_current:
        if sector_weight_current[sector] > 0.40:
            factor = 0.40 / sector_weight_current[sector]

            # scale all stocks inside the sector
            for i in range(x):
                if stocks_chosen_df.iloc[i]["Sector"] == sector:
                    weights[i] *= factor
                    
    # normalizing the weights         
    total = sum(weights)
    return [w / total for w in weights]

# apply the sector cap once
weights = apply_sector_cap(weights)

# max weight per stock
max_stock_cap = 0.15

# created a while loop so it can loop until no stocks violate the 15% stock cap
while True:

    # list to store indices of stocks above the cap
    stocks_above_cap = []

    for i, w in enumerate(weights):
        if w > max_stock_cap:
            stocks_above_cap.append(i)

    # break out of loop if no violations occurred
    if len(stocks_above_cap) == 0:
        break

    # cap violating stocks to max
    for i in stocks_above_cap:
        weights[i] = max_stock_cap

    # calculates the remaining weight after capping
    total_capped = sum(weights[i] for i in stocks_above_cap)
    remaining = 1 - total_capped

    # uncapped stocks
    uncapped = []

    for z in range(x):
        if z not in stocks_above_cap:
            uncapped.append(z)

    # if no weight left or no uncapped stocks, normalize and break
    if remaining <= 0 or len(uncapped) == 0:
        total = sum(weights)
        weights = [w / total for w in weights]
        break

    sum_uncapped = sum(weights[z] for z in uncapped)

    # if uncapped stocks had no weight, use equally-weighted method
    if sum_uncapped == 0:
        equal_weight2 = remaining / len(uncapped)
        for z in uncapped:
            weights[z] = equal_weight2
    else:
        # if it hits the else case, scale weights proportionally
        for z in uncapped:
            weights[z] = (weights[z] / sum_uncapped) * remaining

# apply sector cap again after stock-level cap
weights = apply_sector_cap(weights)

# created a for loop to enforce the minimum weight
for i in range(x):
    if weights[i] < min_weight:
        weights[i] = min_weight

# normalize the weights
total = sum(weights)
weights = [w / total for w in weights]

# add weights to the dataframe
stocks_chosen_df["Weight"] = weights

# printing out the max stock weight and the max sector weight
print(f"Max Stock Weight: {stocks_chosen_df['Weight'].max():.3f}")
print(f"Max Sector Weight: {stocks_chosen_df.groupby('Sector')['Weight'].sum().max():.3f}")


Max Stock Weight: 0.077
Max Sector Weight: 0.231


#### Final Sector Weight Summary and Portfolio Weight Checks:


In [17]:

# dict to store the total weight of each sector
sector_weight_final = {}

# created a for loop to loop through every stock in the final portfolio
for i in range(len(stocks_chosen_df)):

    # sector of current stock
    stock_sector = stocks_chosen_df.iloc[i]["Sector"]

    # weight of current stock
    weight = stocks_chosen_df.iloc[i]["Weight"]
    
    # if this sector has not been added yet, set the value to 0
    if stock_sector not in sector_weight_final:
        
        sector_weight_final[stock_sector] = 0

    # adds the stock's weight to its sector total
    sector_weight_final[stock_sector] += weight

# printing sector-level final weights with the help of a for loop which will loop through the sector totals and printing each one

print("Final Sector Weights:")
for stock_sector in sector_weight_final:
    print(f"{stock_sector} {sector_weight_final[stock_sector]:.4f}")

# printing the min. and max. weight in the portfolio as a sanity check
print(f"Minimum Weight In Portfolio: {stocks_chosen_df['Weight'].min():.4f}")
print(f"Maximum Weight In Portfolio: {stocks_chosen_df['Weight'].max():.4f}")


Final Sector Weights:
Financial Services 0.2307
Technology 0.2008
Industrials 0.1131
Consumer Cyclical 0.0715
Communication Services 0.0644
Real Estate 0.0393
Basic Materials 0.0730
Energy 0.1049
Consumer Defensive 0.0522
Healthcare 0.0502
Minimum Weight In Portfolio: 0.0196
Maximum Weight In Portfolio: 0.0769


### Building the Final Portfolio (Taken Into Account Currency Conversions & Fees):

This cell converts our portfolio weights into actual shares considering stock price, currency conversions, and trading fees. For USD stocks, we convert the prices to CAD using the current CADUSD exchange rate. Trading fees are calculated as the minimum of $2.15 USD flat or 0.001 USD per share as per assignment rules. The final output shows each stock's ticker, price, currency, share count, CAD value, and portfolio weight.

In [18]:

# setting the initial capital and fees
initial_capital = 1_000_000
fee_flat_usd = 2.15
fee_per_share_usd = 0.001

# setting the total budget to the initial capital
total_budget_cad = initial_capital

# extracting the conversion info of "CADUSD=X"
cad_to_usd = yf.Ticker("CADUSD=X")
cad_to_usd_hist = cad_to_usd.history(period="1d")
cadusd = cad_to_usd_hist["Close"].iloc[-1]

# number of selected stocks
x = len(stocks_chosen_df)

# created lists to store values before budget scaling
initial_shares_without_budget = []
cad_value_without_budget = []
cad_fees_without_budget = []

# created a for loop to calculate the number of shares using raw weights
for i in range(x):

    # retrieving necessary info.
    row = stocks_chosen_df.iloc[i]
    price = row["Price"]
    currency = row["Currency"]
    weight = row["Weight"]

    # convert price into CAD if stock is in USD
    if currency == "USD":
        price_cad = price / cadusd
    else:
        price_cad = price

    # calculates the CAD dollars allocated based on weight
    value_in_cad = total_budget_cad * weight

    # calculates the number of shares
    shares = value_in_cad / price_cad

    # calculate USD fees (whichever is cheaper)
    fees_in_usd = min(fee_flat_usd, fee_per_share_usd * shares)

    # converts fees to CAD
    fees_in_cad = fees_in_usd / cadusd

    # storing the initial values (without scaling to budget)
    initial_shares_without_budget.append(shares)
    cad_value_without_budget.append(shares * price_cad)
    cad_fees_without_budget.append(fees_in_cad)

# calculating the total value before adjusting for overspending
total_value_without_budget = sum(cad_value_without_budget)
total_fees_without_budget = sum(cad_fees_without_budget)
total_spent_without_budget = total_value_without_budget + total_fees_without_budget

# used an if statement to see if a stock was overspent on, scale the number of shares downward
if total_spent_without_budget > 0:
    scaling_of_cost = total_budget_cad / total_spent_without_budget
else:
    scaling_of_cost = 1

# final share, value, and fee lists
shares_final = []
cad_value_final = []
cad_fees_final = []

# created a for loop to apply the scaling factor and recalculate everything
for i in range(x):
    
    row = stocks_chosen_df.iloc[i]
    price = row["Price"]
    currency  = row["Currency"]

    # convert price to CAD if needed
    if currency == "USD":
        price_cad = price / cadusd
    else:
        price_cad = price

    # final number of shares
    shares = initial_shares_without_budget[i] * scaling_of_cost

    # calculating the fees again using scaled shares
    fees_in_usd = min(fee_flat_usd, fee_per_share_usd * shares)
    fees_in_cad = fees_in_usd / cadusd

    # convert value of fees to CAD
    value_in_cad = shares * price_cad

    # storing the final results
    shares_final.append(shares)
    cad_value_final.append(value_in_cad)
    cad_fees_final.append(fees_in_cad)

# created variables to store the portfolio totals after scaling
total_value_cad = sum(cad_value_final)
total_fees_cad  = sum(cad_fees_final)
total_spent_cad = total_value_cad + total_fees_cad

# printing those total values out
print(f"Total Invested (CAD): ${total_value_cad:.2f}")
print(f"Total Fees (CAD): ${total_fees_cad:.2f}")
print(f"Total Spent (CAD): ${total_spent_cad:.2f}")

# adding the numbers of shares and the portfolio value of each stock to the main DataFrame
stocks_chosen_df["Shares"] = shares_final
stocks_chosen_df["Value"] = cad_value_final

# constructing the final portfolio
final_portfolio = stocks_chosen_df[["Ticker", "Price", "Currency", "Shares", "Value", "Weight"]].copy()

# outputting the final portfolio
final_portfolio


Total Invested (CAD): $999992.29
Total Fees (CAD): $7.71
Total Spent (CAD): $1000000.00


,Ticker,Price,Currency,Shares,Value,Weight
0,JPM,298.380005,USD,182.896679,76890.765871,0.076891
1,AAPL,266.250000,USD,178.389673,66920.314912,0.066921
2,BAC,51.000000,USD,1070.053178,76890.765871,0.076891
3,WFC,82.400002,USD,662.290183,76890.765871,0.076891
4,CSCO,75.440002,USD,629.589727,66920.314912,0.066921
5,GE,290.619995,USD,92.032986,37684.924812,0.037685
6,AMZN,217.139999,USD,77.883134,23827.687885,0.023828
7,DIS,102.699997,USD,222.478829,32192.727249,0.032193
8,CAT,546.130005,USD,48.974833,37684.924812,0.037685
9,SPG,180.070007,USD,77.411170,19640.098483,0.019640


### Performance Tests (NOT THE FINAL PORTFOLIO)

In [19]:

# edit these dates for the contest dates
contest_start = "2025-11-12"
contest_end = "2025-11-20"

# getting the latest data for the "CADUSD=X" ticker
cadusd_hist = yf.Ticker("CADUSD=X").history(period="1d")
cadusd_end = cadusd_hist["Close"].iloc[-1]

# created a list to store the final CAD value of each value at the end date
final_values_cad = []

# created a for loop to loop through each stock in the final portfolio
for i in range(len(final_portfolio)):

    # retrieving necessary info.
    row = final_portfolio.iloc[i]
    ticker = row["Ticker"]
    shares = row["Shares"]
    currency = row["Currency"]

    # extracting the historical prices for the test window
    hist = yf.Ticker(ticker).history(start=contest_start, end=contest_end)

    # skip if no data is returned
    if hist.empty or "Close" not in hist.columns:
        continue

    # closing price on the last day
    end_price = hist["Close"].iloc[-1]

    # converting the price to CAD
    if currency == "USD":
        price_cad_end = end_price / cadusd_end
    else:
        price_cad_end = end_price

    # add the CAD value to the list
    final_values_cad.append(shares * price_cad_end)

# total portfolio value at the end of the contest
final_value = sum(final_values_cad)

# portfolio total return over the contest window
portfolio_return = (final_value - initial_capital) / initial_capital

# tickers
tsx_symbol = "^GSPTSE"
spx_symbol = "^GSPC"

# extracting ticker data of the S&P and TSX
tsx_hist = yf.Ticker(tsx_symbol).history(start=contest_start, end=contest_end)
spx_hist = yf.Ticker(spx_symbol).history(start=contest_start, end=contest_end)

# calculate TSX total return over the period
tsx_return = tsx_hist["Close"].iloc[-1] / tsx_hist["Close"].iloc[0] - 1

# calculate S&P total return over the period
spx_return = spx_hist["Close"].iloc[-1] / spx_hist["Close"].iloc[0] - 1

# benchmark return (average between the 2)
benchmark_return = (tsx_return + spx_return) / 2

# printing out the results
print("===== MARKET MEET PERFORMANCE CHECKS =====")
print(f"Initial Capital (CAD): {initial_capital:,.2f}")
print(f"Final Value (CAD): {final_value:,.2f}")
print(f"Portfolio Return: {portfolio_return:.4%}")
print(f"Benchmark Return: {benchmark_return:.4%}")

# difference between our portfolio and the benchmark
difference = portfolio_return - benchmark_return
print(f"Difference vs. Benchmark: {difference:.4%}")


===== MARKET MEET PERFORMANCE CHECKS =====
Initial Capital (CAD): 1,000,000.00
Final Value (CAD): 1,016,701.45
Portfolio Return: 1.6701%
Benchmark Return: -2.4143%
Difference vs. Benchmark: 4.0845%


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Jack, Aryan, Samyak